In [1]:
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame
from PIL import Image, ImageTk

In [2]:
# Mendefinisikan direkori citra uang 100k dan 50k
dir_path = "data"
train_dir_path = os.path.join(dir_path, "train")
test_dir_path = os.path.join(dir_path, "test")

train_dir_path_50k = os.path.join(train_dir_path, "50k")
train_dir_path_100k = os.path.join(train_dir_path, "100k")
test_dir_path_50k = os.path.join(test_dir_path, "50k")
test_dir_path_100k = os.path.join(test_dir_path, "100k")

In [3]:
#Perbanyak Gambar Citra dengan rotasi di sudut 90, 180, 270 dan juga flip
def augment_images(base_dir, angles=[90, 180, 270], flips=True):
    for filename in os.listdir(base_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(base_dir, filename)
            img = Image.open(filepath)

            for angle in angles:
                rotated_img = img.rotate(angle)
                rotated_filename = f"{os.path.splitext(filename)[0]}_{angle}.jpg"
                rotated_filepath = os.path.join(base_dir, rotated_filename)
                rotated_img.save(rotated_filepath)
            
            if flips:
                flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
                flipped_filename = f"{os.path.splitext(filename)[0]}_flipped.jpg"
                flipped_filepath = os.path.join(base_dir, flipped_filename)
                flipped_img.save(flipped_filepath)

    print(f"Citra uang di direktori {base_dir} telah diperbanyak dengan augmentasi.")

# # Panggil fungsi augment_images
# augment_images(train_dir_path_50k)
# augment_images(train_dir_path_100k)
# augment_images(test_dir_path_50k)
# augment_images(test_dir_path_100k)

In [4]:
# Melakukan preproses citra dengan threshold dan canny
def preprocess_image(image_path):
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    img_resized = cv.resize(img, (128, 128))  
    _, img_thresholded = cv.threshold(img_resized, 128, 255, cv.THRESH_BINARY)
    img_canny = cv.Canny(img_resized, 25, 200)
    return img_resized, img_thresholded, img_canny

def load_images_from_directory(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(directory, filename)
            img_resized, img_thresholded, img_canny = preprocess_image(filepath)
            images.append(img_canny.flatten())
            labels.append(label)
    return images, labels

# melakukan load citra dari praproses
images_50k, labels_50k = load_images_from_directory(train_dir_path_50k, 0)
images_100k, labels_100k = load_images_from_directory(train_dir_path_100k, 1)

# Menggabungkan data menjadi gambar dan label
X = np.array(images_50k + images_100k)
y = np.array(labels_50k + labels_100k)

# Membagi data latih dan data test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [5]:
# Melatih model k-Nearest Neighbor
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

# Akurasi dari model
y_pred = knn.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        99
           1       0.95      0.95      0.95       116

    accuracy                           0.94       215
   macro avg       0.94      0.94      0.94       215
weighted avg       0.94      0.94      0.94       215

Accuracy: 0.9441860465116279


In [11]:
class CurrencyClassifierApp:
    def __init__(self, master):
        self.master = master
        master.title("PROJECT UAS PENGOLAHAN CITRA DIGITAL")

        # Menambahkan gambar di atas judul
        self.logo_image = Image.open("LOGO UPNVJ.png")  # Ganti dengan path gambar logo yang sesuai
        self.logo_image = self.logo_image.resize((150, 150), Image.LANCZOS)
        self.logo_image_tk = ImageTk.PhotoImage(self.logo_image)
        
        self.logo_label = Label(master, image=self.logo_image_tk)
        self.logo_label.pack(pady=10)
        
        # Judul
        self.title_label = Label(master, text="PROGRAM KLASIFIKASI NOMINAL UANG", font=("Helvetica", 18))
        self.title_label.pack(pady=10)
        
        # Tombol pilih gambar
        self.classify_button = Button(master, text="Pilih Gambar", command=self.classify_image, font=("Helvetica", 14))
        self.classify_button.pack(pady=10)
        
        # Frame untuk gambar yang dipilih
        self.image_frame = Frame(master, width=200, height=200, borderwidth=2, relief="solid")
        self.image_frame.pack(pady=10)
        self.image_label = Label(self.image_frame, text="Tampilan \ngambar yang\ndipilih disini", font=("Helvetica", 14))
        self.image_label.pack()
        
        # Label untuk nama file
        self.file_name_label = Label(master, text="", font=("Helvetica", 14))
        self.file_name_label.pack(pady=10)
        
        # Hasil Klasifikasi
        self.result_label = Label(master, text="", font=("Helvetica", 14))
        self.result_label.pack(pady=10)
        
        # Nama Kelompok
        self.members_label = Label(master, text="Kelompok 8 :\n1. Leli Sawitri \n2. Rahman Ilyas Al Kahfi \n3. Faiz Fristian Nugroho\n4. Farel Bayhaqi", font=("Helvetica", 14), justify="left", anchor="w")
        self.members_label.pack(pady=10)
    
    def classify_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            img_resized, img_thresholded, img_canny = preprocess_image(file_path)
            img_flattened = img_canny.flatten().reshape(1, -1)
            prediction = knn.predict(img_flattened)
            result_text = "Nominal uang tersebut\nadalah Rp 50.000" if prediction == 0 else "Nominal uang tersebut\nadalah Rp 100.000"
            
            img = Image.open(file_path)
            img = img.resize((128, 128), Image.LANCZOS)
            img_tk = ImageTk.PhotoImage(img)
            self.image_label.config(image=img_tk, text="")
            self.image_label.image = img_tk
            
            # Menampilkan relative path
            relative_path = os.path.relpath(file_path)
            self.file_name_label.config(text=f"File: {relative_path}")
            
            self.result_label.config(text=result_text)

root = tk.Tk()
app = CurrencyClassifierApp(root)
root.mainloop()
